In [1]:
from pathlib import Path
import sys
import time
import threading

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import numpy as np
import pandas as pd
%matplotlib inline
import autoclasswrapper as wrapper
import os 

afterall = {}
subfeatures = ['P20', 'P22', 'P23', 'P24']
os.environ['PATH'] += ":/home/chuongdv/Documents/SourceCode/MachineLearning/autoclass-c"
#!touch /home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/autoclass.r-params 
#!echo "xref_class_report_att_list = 0, 1 \nreport_mode = \"data\" \ncomment_data_headers_p = true " >> /home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/autoclass.r-params 
for feature in subfeatures:
    runtimeDir = "/home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/round2/" + feature
    if not Path(runtimeDir).exists():
        os.mkdir(runtimeDir)
    os.chdir(runtimeDir)
    
            # Create object to prepare dataset.
    clust = wrapper.Input()

    # Load datasets from tsv files.
    pathData = "/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/" + feature + ".tsv"
    clust.add_input_data(pathData, "real location")
    clust.add_input_data("/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/P21.tsv", "real location")
    
    # Prepare input data:
    # - create a final dataframe
    # - merge datasets if multiple inputs
    clust.prepare_input_data()

    # Create files needed by AutoClass.
    clust.create_db2_file()
    clust.create_hd2_file()
    clust.create_model_file()
    clust.create_sparams_file(max_duration = 0)
    clust.create_rparams_file()
    #!rm autoclass.r-params
    #!cp /home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/round1/autoclass.r-params .
    
    !rm -f autoclass-run-* *.results-bin


    # Search autoclass in path.
    wrapper.search_autoclass_in_path()

    # Create object to run AutoClass.
    run = wrapper.Run()

    # Prepare run script.
    run.create_run_file()

    # Run AutoClass.
    run.run()
        
    timer = 0
    step = 2
    while not Path(runtimeDir + "/autoclass-run-success").exists():
        timer += step
        sys.stdout.write("\r")
        sys.stdout.write(f"Time: {timer} sec.")
        sys.stdout.flush()
        time.sleep(step)
        if Path("autoclass-run-failure").exists():
            print("error")
            break
            
    if Path(runtimeDir + "/autoclass-run-success").exists():
        results = wrapper.Output()
        results.extract_results()
        results.aggregate_input_data()
#         results.write_cdt()
#         results.write_cdt(with_proba=True)
#         results.write_class_stats()
#         results.write_dendrogram()
        result = pd.read_csv(runtimeDir + '/autoclass_out.tsv', encoding="utf-8", header=0, sep='\t', index_col=0)
        label =  pd.read_csv('/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/label.tsv', encoding='utf-8', header=0, sep='\t', index_col=0)

        label['cluster'] = result['main-class']
        label.to_csv('app.csv', index = True, header=True)


        listH = []
        listApp = []
        listCluster = []
        def fun(x):
            listCluster.append(x['cluster'].values[0])
            listH.append(x['label'].value_counts().max()/x.shape[0])
            listApp.append(x['label'].value_counts().idxmax())


        label.groupby('cluster').apply(fun)

        H = sum(listH)/len(listH)
        print(listH)
        print(listApp)
        print(listCluster)
        print(H)

        dict = {'Cluster': listCluster, 'App': listApp, 'H': listH}

        saveResult = pd.DataFrame(dict)

        saveResult.to_csv(runtimeDir + '/resultCal.csv', index = True, header=True)
        
        afterall.update({feature:H})
        
round1 = pd.DataFrame(list(afterall.items()), index= range(len(subfeatures)))
round1.info()
round1.head(1)
round1.to_csv('/home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/round2/Hp5.csv', index = True, header=True)

2020-04-27 16:30:25 INFO     Reading data file '/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/P20.tsv' as 'real location' with error 0.01
2020-04-27 16:30:25 INFO     Detected encoding: ascii
2020-04-27 16:30:25 INFO     Found 3200 rows and 2 columns
2020-04-27 16:30:25 DEBUG    Checking column names
2020-04-27 16:30:25 DEBUG    Index name 'index'
2020-04-27 16:30:25 DEBUG    Column name 'P20'
2020-04-27 16:30:25 INFO     Checking data format
2020-04-27 16:30:25 INFO     Column 'P20'
2020-04-27 16:30:25 INFO     count    3200.000000
2020-04-27 16:30:25 INFO     mean        0.061129
2020-04-27 16:30:25 INFO     std         1.230634
2020-04-27 16:30:25 INFO     min       -23.639782
2020-04-27 16:30:25 INFO     50%         0.037193
2020-04-27 16:30:25 INFO     max        15.554782
2020-04-27 16:30:25 INFO     ---
2020-04-27 16:30:25 INFO     Reading data file '/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/P21.tsv' as 'real location' with err

Time: 384 sec.

2020-04-27 16:36:49 INFO     Extracting autoclass results
2020-04-27 16:36:49 INFO     Found 3200 cases classified in 21 classes
2020-04-27 16:36:54 INFO     Aggregating input data
2020-04-27 16:36:54 INFO     Writing classes + probabilities .tsv file
2020-04-27 16:36:54 INFO     Reading data file '/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/P22.tsv' as 'real location' with error 0.01
2020-04-27 16:36:54 INFO     Detected encoding: ascii
2020-04-27 16:36:54 INFO     Found 3200 rows and 2 columns
2020-04-27 16:36:54 DEBUG    Checking column names
2020-04-27 16:36:54 DEBUG    Index name 'index'
2020-04-27 16:36:54 DEBUG    Column name 'P22'
2020-04-27 16:36:54 INFO     Checking data format
2020-04-27 16:36:54 INFO     Column 'P22'
2020-04-27 16:36:54 INFO     count    3200.000000
2020-04-27 16:36:54 INFO     mean       -0.023030
2020-04-27 16:36:54 INFO     std         0.977295
2020-04-27 16:36:54 INFO     min        -8.773719
2020-04-27 16:36:54 INFO     50%   

[0.07297297297297298, 0.18072289156626506, 0.08187134502923976, 0.05791505791505792, 0.20853080568720378, 0.09210526315789473, 0.09292035398230089, 0.07929515418502203, 0.13068181818181818, 0.25663716814159293, 1.0, 0.0989010989010989, 0.1095890410958904, 0.12857142857142856, 0.9056603773584906, 0.1951219512195122, 0.3333333333333333, 0.2, 0.1875, 0.25, 0.2]
['APPLE', 'CONTENT_FLASH', 'APPLE_ICLOUD', 'GOOGLE', 'DROPBOX', 'HTTP_CONNECT', 'SPOTIFY', 'WINDOWS_UPDATE', 'CLOUDFLARE', 'OFFICE_365', 'IP_ICMP', 'DNS', 'EBAY', 'SSL', 'DNS', 'WHATSAPP', 'DNS', 'HTTP', 'SKYPE', 'HTTP', 'MS_ONE_DRIVE']
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
0.23153952672852962


2020-04-27 16:36:55 INFO     AutoClass C executable found in /home/chuongdv/Documents/SourceCode/MachineLearning/autoclass-c/autoclass
2020-04-27 16:36:55 INFO     Writing run file
2020-04-27 16:36:55 INFO     AutoClass C executable found in /home/chuongdv/Documents/SourceCode/MachineLearning/autoclass-c/autoclass
2020-04-27 16:36:55 INFO     AutoClass C version: AUTOCLASS C (version 3.3.6unx)
2020-04-27 16:36:55 INFO     Running clustering...


...

[0.06569343065693431, 0.23404255319148937, 0.08695652173913043, 0.09022556390977443, 0.09316770186335403, 0.1375, 0.18110236220472442, 0.08387096774193549, 0.10344827586206896, 0.20930232558139536, 0.17391304347826086, 0.1509433962264151, 0.13636363636363635, 0.13157894736842105, 0.10084033613445378, 1.0, 0.09523809523809523, 0.19230769230769232, 0.12, 0.1, 0.18947368421052632, 0.10714285714285714, 0.14285714285714285, 0.1, 0.2702702702702703, 0.16, 0.4375, 1.0, 0.13157894736842105, 0.13636363636363635, 0.1875, 0.1111111111111111, 0.6666666666666666, 0.7142857142857143]
['INSTAGRAM', 'CONTENT_FLASH', 'HTTP_CONNECT', 'DNS', 'INSTAGRAM', 'APPLE', 'CLOUDFLARE', 'CLOUDFLARE', 'INSTAGRAM', 'OFFICE_365', 'CONTENT_FLASH', 'APPLE_ICLOUD', 'HTTP_CONNECT', 'HTTP', 'APPLE_ITUNES', 'IP_ICMP', 'SPOTIFY', 'DNS', 'DROPBOX', 'MSN', 'MS_ONE_DRIVE', 'NETFLIX', 'WINDOWS_UPDATE', 'YOUTUBE', 'FACEBOOK', 'OFFICE_365', 'APPLE_ICLOUD', 'DNS', 'EBAY', 'GOOGLE', 'SKYPE', 'EBAY', 'SPOTIFY', 'DROPBOX']
[1, 2, 3, 

2020-04-27 16:55:29 INFO     AutoClass C executable found in /home/chuongdv/Documents/SourceCode/MachineLearning/autoclass-c/autoclass
2020-04-27 16:55:29 INFO     Writing run file
2020-04-27 16:55:29 INFO     AutoClass C executable found in /home/chuongdv/Documents/SourceCode/MachineLearning/autoclass-c/autoclass
2020-04-27 16:55:29 INFO     AutoClass C version: AUTOCLASS C (version 3.3.6unx)
2020-04-27 16:55:29 INFO     Running clustering...


Time: 500 sec.

2020-04-27 17:03:50 INFO     Extracting autoclass results
2020-04-27 17:03:50 INFO     Found 3200 cases classified in 29 classes
2020-04-27 17:03:54 INFO     Aggregating input data
2020-04-27 17:03:55 INFO     Writing classes + probabilities .tsv file


[0.09763313609467456, 0.07013574660633484, 0.07627118644067797, 0.245136186770428, 0.10526315789473684, 0.1076923076923077, 0.11797752808988764, 0.3246753246753247, 0.15384615384615385, 1.0, 0.12222222222222222, 0.13095238095238096, 0.325, 0.16666666666666666, 0.14666666666666667, 0.2631578947368421, 0.2028985507246377, 0.2459016393442623, 0.18181818181818182, 0.24528301886792453, 0.14545454545454545, 0.1346153846153846, 0.15384615384615385, 0.8837209302325582, 0.13043478260869565, 0.125, 0.17647058823529413, 0.2962962962962963, 0.5555555555555556]
['DROPBOX', 'APPLE_ICLOUD', 'CLOUDFLARE', 'CONTENT_FLASH', 'HTTP_PROXY', 'CLOUDFLARE', 'SPOTIFY', 'DNS', 'MS_ONE_DRIVE', 'IP_ICMP', 'WINDOWS_UPDATE', 'CLOUDFLARE', 'OFFICE_365', 'HTTP_CONNECT', 'EBAY', 'AMAZON', 'APPLE_ICLOUD', 'APPLE', 'HTTP', 'HTTP', 'SKYPE', 'SSL', 'HTTP_PROXY', 'DNS', 'CONTENT_FLASH', 'GMAIL', 'EBAY', 'APPLE_ICLOUD', 'INSTAGRAM']
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 